In [9]:
#Import the library 
import pandas as pd 
import os

In [6]:
# Read the csv file 
df=pd.read_csv("/content/drive/My Drive/train.csv")

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
df.head()
df.shape

(20800, 5)

In [10]:
os.getcwd()

'/content'

SyntaxError: ignored

In [12]:
# Drop the NA values 
df=df.dropna()
df.shape

(18285, 5)

In [13]:
## Get the independent features 
X=df.drop('label',axis=1)

In [14]:
##Now get Dependent feature 
y=df['label']

In [15]:
df.shape

(18285, 5)

In [16]:
X.shape

(18285, 4)

In [17]:
y.shape

(18285,)

In [18]:
import tensorflow as tf 

In [19]:
tf.__version__

'2.3.0'

In [20]:
from tensorflow.keras.preprocessing.text import one_hot 
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM 
from tensorflow.keras.layers import Dense # This should have one node greater than 0.5 Fake , less than 0.5 not fake

In [21]:
#Now give the Vocabulary size 
voc_size = 5000

In [22]:
messages = X.copy()

In [23]:
messages.reset_index(inplace=True)

In [24]:
messages.head(9)

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
5,5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi..."
6,7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi..."
7,9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...
8,10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that..."


In [29]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [30]:
## Dataset preprocessing 
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]
for i in range(0,len(messages)):
    review= re.sub('[^a-zA-Z]',' ',messages['title'][i])
    review= review.lower()
    review= review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [31]:
len(corpus)

18285

In [32]:
# Giving index based in Vocabulary size
onehot_repr=[one_hot(c,voc_size) for c in corpus]

In [33]:
#Padding representation
sent_length=20 
embedded_docs= pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2966  905 2853]
 [   0    0    0 ...  418  389 2509]
 [   0    0    0 ... 2197 3481  774]
 ...
 [   0    0    0 ... 3762 2604 4236]
 [   0    0    0 ... 3979 4361 4430]
 [   0    0    0 ... 1857 1564 3115]]


In [34]:
# Creating model 
embedding_vector_features=40 
model=Sequential() #This object is instance of class , this can be checked by the below function 
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length)) #This is to be done with embedding layer 
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [35]:
isinstance(model,Sequential)

True

In [36]:
len(embedded_docs),y.shape

(18285, (18285,))

In [37]:
import numpy as np
X_final= np.array(embedded_docs)
y_final= np.array(y)

In [40]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [41]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_final,y_final,test_size=0.33,random_state=42)

In [42]:
#Training the model 
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 6s 34ms/step - loss: 0.3325 - accuracy: 0.8436 - val_loss: 0.1943 - val_accuracy: 0.9157
Epoch 2/10
192/192 [==============================] - 6s 31ms/step - loss: 0.1437 - accuracy: 0.9430 - val_loss: 0.2004 - val_accuracy: 0.9183
Epoch 3/10
192/192 [==============================] - 6s 31ms/step - loss: 0.1009 - accuracy: 0.9621 - val_loss: 0.2143 - val_accuracy: 0.9171
Epoch 4/10
192/192 [==============================] - 6s 31ms/step - loss: 0.0704 - accuracy: 0.9749 - val_loss: 0.2478 - val_accuracy: 0.9147
Epoch 5/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0504 - accuracy: 0.9825 - val_loss: 0.2783 - val_accuracy: 0.9176
Epoch 6/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0317 - accuracy: 0.9896 - val_loss: 0.3827 - val_accuracy: 0.9157
Epoch 7/10
192/192 [==============================] - 6s 31ms/step - loss: 0.0223 - accuracy: 0.9935 - val_loss: 0.3582 - val_accuracy: 0.9145

In [43]:
# Performance metrics and Accuracy 
y_pred=model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [46]:
from sklearn.metrics import confusion_matrixb

In [47]:
confusion_matrix(y_test,y_pred)

array([[3113,  306],
       [ 220, 2396]])

In [48]:
from sklearn.metrics import accuracy_score

In [49]:
accuracy_score(y_test,y_pred) #this is matching with the model fit and validatio

0.9128417564208782

In [57]:
from tensorflow.keras.layers import Dropout
# Adding dropouts
embedding_vector_features=40 
model=Sequential() #This object is instance of class , this can be checked by the below function 
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length)) #This is to be done with embedding layer 
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid')) #Addition of this layer is for classification problem
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
dropout_2 (Dropout)          (None, 20, 40)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [58]:
#Training the model 
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 7s 34ms/step - loss: 0.3598 - accuracy: 0.8292 - val_loss: 0.2104 - val_accuracy: 0.9082
Epoch 2/10
192/192 [==============================] - 6s 32ms/step - loss: 0.1601 - accuracy: 0.9347 - val_loss: 0.1914 - val_accuracy: 0.9198
Epoch 3/10
192/192 [==============================] - 6s 32ms/step - loss: 0.1202 - accuracy: 0.9558 - val_loss: 0.2204 - val_accuracy: 0.9213
Epoch 4/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0951 - accuracy: 0.9664 - val_loss: 0.2211 - val_accuracy: 0.9196
Epoch 5/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0732 - accuracy: 0.9730 - val_loss: 0.2482 - val_accuracy: 0.9175
Epoch 6/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0569 - accuracy: 0.9798 - val_loss: 0.2761 - val_accuracy: 0.9183
Epoch 7/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0400 - accuracy: 0.9872 - val_loss: 0.3218 - val_accuracy: 0.9157

In [59]:
y_pred=model.predict_classes(X_test)

In [61]:
check=accuracy_score(y_test,y_pred)
check

0.9194697597348799